# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import nltk
#from custom_transformer import StartingVerbExtractor
import pandas as pd
from sqlalchemy import create_engine, text
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer
import re
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.base import BaseEstimator,TransformerMixin
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import GridSearchCV
import pickle
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!pip install transformer

  Could not find a version that satisfies the requirement transformer (from versions: )
No matching distribution found for transformer


In [6]:
# load data from database
engine = create_engine('sqlite:///Message.db')
#Reading from SQL table
df = pd.read_sql_table('Message', engine) 
# Dropping NA values.
df=df.dropna()

#Splitting the variable in dependent and independent variable X
X=df[['message']]
# Y Variable
Y=df.drop(['message','original','genre','id'],axis=1)


In [7]:
#Changing list to arrays
X=np.array(X).reshape(-1)
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       'shikarpur: Volunteers needed, food and medical help needed',
       'Dadu: Needs water, food, medicines. boats',
       'Umer kot: Sindh: Shelter and Camp needed as people are stranded due to flood:'], dtype=object)

### 2. Write a tokenization function to process your text data

In [11]:


def tokenize(text):
    """Normalize, tokenize and lemmatize text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    lemmatized: list of strings. List containing normalized and lemmatized word tokens
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # Stem word tokens and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return lemmatized   

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
'''Creating the pipeline and using MultiOutout classifier '''

pipeline = Pipeline([('vectorizer',CountVectorizer(tokenizer=tokenize)),
                     ('tfid', TfidfTransformer()),
                     ('clf',MultiOutputClassifier(DecisionTreeClassifier()))                        
                    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
#Splitting the dataset in test and train set
X_train, X_test, y_train, y_test = train_test_split(X,Y)
#Train the pipeline
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [14]:
#Printing the shape of x_train and y_train
print(X_train.shape)
print(y_train.shape)

(7623,)
(7623, 36)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred[:,1]

NameError: name 'y_pred' is not defined

In [15]:
#creating new list
metrics_list_all=[]
#storing the columns name category name variable
category_names=y_test.columns
#Predicting the outcomes from the list.
y_pred=pipeline.predict(X_test)

for col in range(y_test.shape[1]):
    #Creating the test variable of each column
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    #Predicting the test variable of each column
    predicted=y_pred[:,col].astype('int').astype('str')
    #Accuracy
    accuracy = accuracy_score(tested, predicted)
    #precision
    precision=precision_score(tested, predicted,average='micro')
    #recall
    recall = recall_score(tested, predicted,average='micro')
    #F1-score
    f_1 = f1_score(tested, predicted,average='micro')
    metrics_list=[accuracy,precision,recall,f_1]
    metrics_list_all.append(metrics_list)
metrics_df=pd.DataFrame(metrics_list_all,index=category_names,columns=["Accuracy","Precision","Recall","F_1"])
print(metrics_df)

                        Accuracy  Precision    Recall       F_1
related                 0.570810   0.570810  0.570810  0.570810
request                 0.567270   0.567270  0.567270  0.567270
offer                   0.998820   0.998820  0.998820  0.998820
aid_related             0.541306   0.541306  0.541306  0.541306
medical_help            0.905980   0.905980  0.905980  0.905980
medical_products        0.939024   0.939024  0.939024  0.939024
search_and_rescue       0.964988   0.964988  0.964988  0.964988
security                0.974430   0.974430  0.974430  0.974430
military                0.992526   0.992526  0.992526  0.992526
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.872541   0.872541  0.872541  0.872541
food                    0.752164   0.752164  0.752164  0.752164
shelter                 0.827695   0.827695  0.827695  0.827695
clothing                0.981511   0.981511  0.981511  0.981511
money                   0.976790   0.976

In [16]:
#Generating Classification Report
y_labels=y_test.columns
x=0
for column in y_test.columns:  
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    predicted=y_pred[:,col].astype('int').astype('str')
    
    print(classification_report(tested, predicted))
    x=x+1

             precision    recall  f1-score   support

          0       0.68      0.70      0.69      1699
          1       0.37      0.35      0.36       843

avg / total       0.58      0.58      0.58      2542

             precision    recall  f1-score   support

          0       0.68      0.70      0.69      1699
          1       0.37      0.35      0.36       843

avg / total       0.58      0.58      0.58      2542

             precision    recall  f1-score   support

          0       0.68      0.70      0.69      1699
          1       0.37      0.35      0.36       843

avg / total       0.58      0.58      0.58      2542

             precision    recall  f1-score   support

          0       0.68      0.70      0.69      1699
          1       0.37      0.35      0.36       843

avg / total       0.58      0.58      0.58      2542

             precision    recall  f1-score   support

          0       0.68      0.70      0.69      1699
          1       0.37      0.35 

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# Getting parameter From pipeline
sorted(pipeline.get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__presort',
 'clf__estimator__random_state',
 'clf__estimator__splitter',
 'clf__n_jobs',
 'memory',
 'steps',
 'tfid',
 'tfid__norm',
 'tfid__smooth_idf',
 'tfid__sublinear_tf',
 'tfid__use_idf',
 'vectorizer',
 'vectorizer__analyzer',
 'vectorizer__binary',
 'vectorizer__decode_error',
 'vectorizer__dtype',
 'vectorizer__encoding',
 'vectorizer__input',
 'vectorizer__lowercase',
 'vectorizer__max_df',
 'vectorizer__max_features',
 'vectorizer__min_df',
 'vectorizer__ngram_range',
 'vectorizer__preprocessor',
 'vectorizer__stop_words',
 'vectorizer__strip_accents',
 'vectorizer__token_pa

In [19]:
#Optimising the hyperparameter Values
from sklearn.model_selection import GridSearchCV

parameters = {
             #  'vect__stop_words': ('english', None),
                'clf__estimator__criterion':['gini','entropy'],
                'clf__estimator__max_features': ['auto','sqrt','log2'],
                'clf__estimator__min_samples_split': [2, 5]
             }
cv = GridSearchCV(pipeline,param_grid=parameters)
cv.fit(X_train,y_train)













GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_features': ['auto', 'sqrt', 'log2'], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
#Checking the accuracy
(y_test==y_pred).mean().mean()

0.90087638779613599

In [87]:
compare_per_DF=pd.concat([metrics_df_new,metrics_df],axis=1)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
y_labels=y_test.columns
x=0
upd_y_pred= upd_pipeline.predict(X_test)
for column in y_test.columns:  
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    predicted=upd_y_pred[:,col].astype('int').astype('str')
    
    print(classification_report(tested, predicted))
    x=x+1
    

NameError: name 'upd_pipeline' is not defined

In [23]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            print(pos_tags)
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
            return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [24]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import FeatureUnion
#from custom_transformer import StartingVerbExtractor
upd_pipeline = Pipeline([
        ('features', FeatureUnion ([
            
            ('text_pipeline', Pipeline ([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
                                        ])),      
             ('starting_verb', StartingVerbExtractor())   
                                    ])),        
        
        ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier))
        ])


upd_pipeline.fit(X_train, y_train)

    


[('water', 'NN'), ('mariani', 'CD'), ('carrefour', 'NN')]
[('dollar', 'NN'), ('buy', 'VB'), ('sell', 'JJ'), ('pleas', 'NNS')]
[('social', 'JJ'), ('worker', 'NN'), ('need', 'MD'), ('help', 'VB'), ('victim', 'VB'), ('port', 'NN'), ('au', 'NN'), ('princ', 'NN'), ('gressier', 'NN'), ('leogan', 'JJ'), ('petit', 'JJ'), ('goav', 'NN'), ('jacmel', 'NN')]
[('would', 'MD'), ('like', 'VB'), ('know', 'VB'), ('whrere', 'RB'), ('find', 'VB'), ('scholarship', 'NN')]
[('port', 'NN'), ('princ', 'NN'), ('relationship', 'NN'), ('earthquak', 'NN')]
[('brother', 'NN'), ('find', 'VB'), ('inform', 'NN')]
[('thank', 'NN'), ('help', 'NN'), ('us', 'PRP'), ('help', 'VB'), ('n', 'VB'), ('ap', 'JJ'), ('year', 'NN')]
[('give', 'VB'), ('way', 'NN'), ('live', 'JJ')]
[('would', 'MD'), ('like', 'VB'), ('know', 'VB'), ('decid', 'NN'), ('want', 'VBP'), ('leav', 'JJ'), ('school', 'NN'), ('port', 'NN'), ('au', 'NN'), ('princ', 'NN'), ('go', 'VBP'), ('countrysid', 'NN')]
[('port', 'NN'), ('de', 'FW'), ('paix', 'FW'), ('mone

TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

### 9. Export your model as a pickle file

In [31]:
pickle.dump(pipeline,open( "Classifier.pkl", "wb" ))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [34]:
pwd

'/home/workspace'